#**Mô tả bài toán**
Sinh viên cần triển khai các chiến lược tìm kiếm để giúp Pacman thu thập tất cả các điểm thức ăn (food points) và thoát khỏi mê cung qua cổng thoát (exit gate) được chỉ định.  
*   **Dữ liệu đầu vào:** Đường dẫn đến tệp bố cục mê cung (layout file).  
*   **Dữ liệu đầu ra:** Danh sách các hành động (North, East, West, South, Stop) và tổng chi phí.

##Cấu trúc mê cung:


*   % → tường hoặc chướng ngại vật.
*   P → vị trí ban đầu của Pacman.
*   . → điểm thức ăn (có thể có nhiều điểm).
*   O → bánh ma thuật (magical pie): Pacman có thể ăn tường trong 5 bước sau khi ăn bánh.
*  (khoảng trống) → ô trống có thể di chuyển.
*   G → vị trí ban đầu của ma.
*   E → cổng thoát, nơi Pacman phải đến sau khi thu thập hết thức ăn.
##Các quy tắc bổ sung:
*   Ma di chuyển chỉ theo chiều ngang, khi gặp tường thì đổi hướng.
*   Ở bốn góc mê cung (trên-trái, trên-phải, dưới-trái, dưới-phải) có cổng dịch chuyển (teleport): Pacman chạm vào một góc có thể dịch chuyển đến bất kỳ góc nào khác.
*   Sau mỗi 30 bước, mê cung xoay phải 90°.


## 🧩 Yêu cầu
1. Mô hình hóa bài toán dưới dạng **không gian trạng thái**  
2. Cài đặt **thuật toán A\*** với **hàm heuristic**  
3. Xây dựng **giao diện (GUI)** bằng `pygame`  
4. Hỗ trợ **chế độ thủ công** (phím mũi tên) và **chế độ tự động** (A\*)  
5. Áp dụng **lập trình hướng đối tượng (OOP)** và **tái sử dụng A\*** từ Task 1  
6. Phân tích **độ phức tạp thời gian và không gian**



























## Các thư viện cần thiết

In [23]:
import random  # Dùng để chọn ngẫu nhiên điểm dịch chuyển trong chế độ chơi thủ công (Manual)
import os  # Giúp tìm đường dẫn tới file 'pacman.txt', đảm bảo code chạy được trên mọi máy
import pygame  # Thư viện chính để vẽ, hiển thị hình ảnh, cửa sổ game và xử lý điều khiển
import heapq  # Cung cấp cấu trúc "hàng đợi ưu tiên", cách để A* luôn chọn được node tốt nhất
import time  # Dùng để bấm giờ, giới hạn thời gian chạy của thuật toán A* để tránh treo máy
import numpy as np  # Thư viện toán học mạnh mẽ, ở đây dùng chủ yếu để xoay ma trận (mê cung) cho dễ
import traceback  # Khi có lỗi nghiêm trọng, thư viện này giúp in ra chi tiết lỗi để dễ sửa

# Define current directory path
CURRENT_DIR = os.path.dirname(os.path.abspath(__file__ if '__file__' in locals() else 'pacman.ipynb'))

# Dùng một cấu trúc dữ liệu đặc biệt (hàng đợi hai đầu) để thuật toán BFS chạy nhanh hơn
from collections import deque

In [24]:
def bfs_distance(maze, start, end):
    """
    Hàm này tính khoảng cách ngắn nhất giữa 2 điểm bằng thuật toán BFS.
    Đặc điểm: Nó "coi như không có tường" khi tính toán.
    Mục đích: Dùng làm một hàm heuristic phụ, nó luôn đưa ra một ước tính "lạc quan",
    không bao giờ đoán xa hơn thực tế, đảm bảo tính đúng đắn của A*.
    """
    if start == end:
        return 0

    queue = deque([(start, 0)])  # Hàng đợi chứa (vị trí, khoảng cách)
    visited = {start}  # Tập hợp các điểm đã ghé thăm để không đi lại

    while queue:
        (x, y), dist = queue.popleft()

        if (x, y) == end:
            return dist # Tìm thấy đích, trả về khoảng cách

        # Xét 4 hướng xung quanh
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy

            # Nếu điểm tiếp theo hợp lệ và chưa được ghé thăm
            if (0 <= nx < maze.width and 0 <= ny < maze.height and (nx, ny) not in visited):
                visited.add((nx, ny))
                queue.append(((nx, ny), dist + 1))

    return float('inf') # Nếu không tìm thấy đường đi

**CLASS STATE**: Dùng để lưu trữ State, trạng thái của pacman.
Hàm gồm các thuộc tính:

*   **pos:** Tọa độ hiện tại của Pacman trong mê cung (x, y).
*  **dots:** Các điểm thức ăn còn lại trong mê cung.
*   **pies:** các bánh phép còn lại trong mê cung
*   **pie_steps:** số bước đi xuyên tường còn lại.
*   **can_tele**: trạng thái teleport để tránh việc pacman khi vô góc sẽ bị dịch chuyển qua lại vô hạn




In [25]:
class State:
    """
    Đây là "bản tóm tắt" của game tại một thời điểm nhất định.
    Nó chứa tất cả thông tin quan trọng: Pacman ở đâu, các chấm còn lại,
    vị trí ma, năng lượng còn bao nhiêu, v.v.
    Việc dùng tuple và frozenset là chủ đích: chúng là các kiểu dữ liệu "bất biến",
    giúp cho State có thể được 'băm' (hash), một yêu cầu bắt buộc của thuật toán A*.
    """
    def __init__(self, pos, dots, pies, ghost_positions, ghost_directions, pie_steps=0, can_tele=True, step_number=0):
        self.pos = (int(pos[0]), int(pos[1])) # Vị trí Pacman
        self.dots = frozenset((int(x), int(y)) for x, y in dots) # Các chấm còn lại (dùng frozenset cho nhanh)
        self.pies = frozenset((int(x), int(y)) for x, y in pies) # Các bánh năng lượng còn lại
        self.ghost_positions = tuple((int(x), int(y)) for x, y in ghost_positions) # Vị trí ma (dùng tuple để giữ thứ tự)
        self.ghost_directions = tuple(sorted(ghost_directions.items())) # Hướng đi của từng con ma
        self.pie_steps = int(pie_steps) # Số bước đi xuyên tường còn lại
        self.can_tele = bool(can_tele) # Liệu có thể dịch chuyển trong bước này không
        self.step_number = int(step_number)

    # Các hàm get_... để lấy thông tin từ State ra ngoài
    def get_pos(self): return self.pos
    def get_dots(self): return set(self.dots)
    def get_pies(self): return set(self.pies)

    def get_ghost_positions(self):
        # Trả về một list (copy)
        return list(self.ghost_positions)

    def get_ghost_directions(self):
        # Chuyển lại thành dict
        return dict(self.ghost_directions)

    def get_pieSteps(self): return self.pie_steps
    def get_canTele(self): return self.can_tele

    def __eq__(self, other):
        """
        Hàm này định nghĩa khi nào hai State được coi là "bằng nhau".
        Cần thiết để kiểm tra xem mình đã từng ở trạng thái này chưa.
        """
        if not isinstance(other, State): return False
        # So sánh trực tiếp các tuple
        return (self.pos == other.pos and
                self.dots == other.dots and
                self.pies == other.pies and
                self.ghost_positions == other.ghost_positions and # So sánh tuple
                self.ghost_directions == other.ghost_directions and # So sánh tuple
                self.pie_steps == other.pie_steps and
                self.can_tele == other.can_tele and
                self.step_number == other.step_number)

    def __hash__(self):
        """
        Hàm này tạo ra một "định danh" (một con số) duy nhất cho mỗi State.
        Giúp cho việc lưu trữ và tìm kiếm State trong 'explored' set của A* nhanh hơn.
        """
        # Hash tất cả các thuộc tính (tuple hash được)
        return hash((self.pos, self.dots, self.pies, self.ghost_positions,
                     self.ghost_directions, self.pie_steps, self.can_tele,
                 self.step_number))

    def __str__(self):
        """Hàm định nghĩa cách State được in ra màn hình, giúp gỡ lỗi dễ hơn."""
        ghost_pos_str = str(list(self.ghost_positions))
        ghost_dirs_str = str(dict(self.ghost_directions))
        return (
            f"Pos:{self.pos}, Dots:{len(self.dots)}, Pies:{len(self.pies)}, "
            f"Ghosts:{ghost_pos_str}, Dirs:{ghost_dirs_str}, "
            f"PieSteps:{self.pie_steps}, CanTele:{self.can_tele}"
        )

#**PacmanProblemAdapter Class**
Lớp này đóng vai trò như **bộ chuyển đổi (Adapter)**, giúp **bài toán Pac-Man** tương thích với **A\* Solver**.  
Nó sẽ "gói" (`wrap`) lại lớp `Problem` gốc và cung cấp các phương thức mà **AStarSolver** yêu cầu.

In [26]:
class PacmanProblemAdapter:
    """
    Lớp Adapter để làm cho Problem của Pac-Man tương thích với AStarSolver.
    Nó sẽ "gói" Problem gốc và cung cấp các phương thức mà AStarSolver yêu cầu.
    """
    def __init__(self, pacman_problem):
        self.problem = pacman_problem
        # "Làm giàu" trạng thái ban đầu với step_number = 0
        initial = self.problem.get_initial_state()
        self.initial_state = State(
            initial.pos, initial.dots, initial.pies, initial.ghost_positions,
            initial.get_ghost_directions(), initial.pie_steps, initial.can_tele,
            step_number=0
        )

    def is_goal(self, state):
        # AStarSolver gọi is_goal(state), chúng ta gọi hàm gốc với state và step_number từ state
        return self.problem.is_goal_state(state, state.step_number)

    def get_successors(self, state):
        """
        Đây là phần quan trọng nhất.
        Nó tạo ra các trạng thái kế tiếp và đảm bảo mỗi trạng thái mới
        đều có step_number được cập nhật.
        """
        successors = []
        current_step_number = state.step_number
        current_rotation_level = current_step_number // 30

        # 1. Lấy các hành động hợp lệ từ trạng thái hiện tại
        actions = self.problem.get_actions(state, current_rotation_level)

        # 2. Với mỗi hành động, tạo ra trạng thái kế tiếp
        for action in actions:
            next_step_number = current_step_number + 1

            # Sử dụng logic của Problem gốc để tính toán trạng thái tiếp theo
            neighbor_state = self.problem.get_neighbor_state(state, action, next_step_number)

            if neighbor_state is None:
                continue

            # Xử lý xoay mê cung nếu cần
            if next_step_number % 30 == 0:
                neighbor_state = self.problem.rotate_state(neighbor_state, current_rotation_level)
                if neighbor_state is None:
                    continue

            # 3. "Làm giàu" trạng thái kế tiếp với step_number mới
            final_succ_state = State(
                neighbor_state.pos, neighbor_state.dots, neighbor_state.pies,
                neighbor_state.ghost_positions, neighbor_state.get_ghost_directions(),
                neighbor_state.pie_steps, neighbor_state.can_tele,
                step_number=next_step_number # <-- Cập nhật step_number
            )
            successors.append((action, final_succ_state))

        return successors

    def cost(self, state, succ):
        # Chi phí mỗi bước đi luôn là 1
        return 1

Vì hàm heuristic của AStarSolver cũng chỉ nhận state,
chúng ta cần một hàm "bọc" (wrapper) nhỏ để gọi hàm heuristic_custom với đúng tham số.

In [27]:
def heuristic_wrapper(problem_instance):
    """
    Tạo một hàm heuristic chỉ nhận `state` làm đầu vào,
    để tương thích với AStarSolver.
    """
    return lambda state: problem_instance.heuristic_custom(state, state.step_number)

##**NODE CLASS:**  
Dùng lưu trữ nhiều thuộc tính hơn state, chứa cả state, là 1 node trên không gian trạng thái bài toán.

- Hàm khởi tạo Node gồm các thuộc tính:
    + **state:** trạng state của pacman
    +  **parent:** cha của node để có thể truy vết đường đi sau khi tìm thấy đích (mặc định là None).
    + **action:** hành động di chuyển của node (North, East, West, South) dùng để biết Pacman đã đi theo hướng nào (mặc định là None).
    +  **g:** cost đường đi đến node hiện tại, tính từ vị trí bắt đầu (mặc định là 0)
    + **h:** giá trị của hàm heuristic, để tính chi phí ước lượng đường đi đến node đích, tính từ node hiện tại (mặc định là 0).
    + **f:** tổng giá trị g và h của node hiện tại

**\*** Không cần phải khai báo phương thức hash, str, eq ở trong node, vì node chứa state (state đã chứa hết phương thức đó rồi, kiểm tra explored ở trên State)

In [28]:
class Node:
    """
    Đại diện cho một nút trong cây tìm kiếm A* (A-star search).

    Mỗi Node lưu trữ:
        - state: trạng thái (đối tượng State)
        - parent: Node cha (dùng để truy ngược đường đi)
        - action: hành động dẫn từ cha -> state hiện tại ('U','D','L','R', ...)
        - g: chi phí thực tế từ gốc -> node hiện tại (path cost)
        - h: giá trị heuristic (ước lượng chi phí còn lại đến goal)
        - f: tổng chi phí dự đoán = g + h

    Thuộc tính __lt__ giúp so sánh node trong hàng đợi ưu tiên (priority queue).
    """

    def __init__(self, state, parent=None, action=None, g=0, h=0):
        """
        Khởi tạo Node mới.
        Args:
            state  : trạng thái hiện tại (State)
            parent : node cha (Node) — None nếu là node gốc
            action : hành động dẫn từ cha đến node này ('U','D','L','R')
            g      : chi phí thực tế từ gốc tới đây
            h      : heuristic (ước lượng chi phí còn lại)
        """
        self.state = state
        self.parent = parent
        self.action = action
        self.g = g
        self.h = h
        self.f = g + h  # tổng chi phí dự đoán (evaluation function)

    def __lt__(self, other):
        """
        So sánh 2 node để dùng trong hàng đợi ưu tiên (heapq).
        - Ưu tiên node có f nhỏ hơn.
        - Nếu f bằng nhau: ưu tiên node có g nhỏ hơn (đi sâu ít hơn).
        """
        return self.f < other.f or (self.f == other.f and self.g < other.g)

    def reconstruct_path(self):
        """
        Truy ngược đường đi từ node hiện tại về node gốc.
        Returns:
            List[State] — danh sách trạng thái theo thứ tự từ start -> goal.

        Ý tưởng:
          - Duyệt ngược bằng parent.
          - Thu thập tất cả state trên đường đi.
          - Đảo ngược lại vì ta duyệt từ goal về start.
        """
        path, node = [], self
        # Bỏ qua node cuối cùng vì nó không có hành động dẫn đến nó
        while node and node.parent:
            path.append(node.action) # <-- SỬA THÀNH LẤY ACTION
            node = node.parent
        path.reverse() # Dùng reverse() thay vì reversed() cho list
        return path

#Class: Maze
Lớp `Maze` chịu trách nhiệm **đọc bản đồ mê cung** từ file `.txt`,  
xử lý dữ liệu để xác định **vị trí Pac-Man, điểm ăn, bánh, ma, lối ra**,  
và cung cấp các **phương thức** để làm việc với bản đồ.

In [29]:
class Maze:
    """
    Lớp này chịu trách nhiệm đọc file .txt và phân tích cấu trúc của mê cung.
    Nó biết đâu là tường, đâu là đường đi, vị trí ban đầu của các vật thể, v.v.
    """
    def __init__(self, file_path):
        self.grid = []
        self.height = 0
        self.width = 0
        self.dots = set()
        self.pies = set()
        self.initial_position = (-1, -1)
        self.exit_position = (-1, -1)
        self.ghost_positions = []

        with open(file_path, "r") as f:
            lines = f.readlines()
            maze_lines = [list(line.strip()) for line in lines if line.strip() and not line.strip().endswith('.txt')]

        self.grid = np.array(maze_lines, dtype=str)
        self.height, self.width = self.grid.shape

        original_grid = np.copy(self.grid)
        empty_spaces = [ (x, y) for y in range(self.height) for x in range(self.width) if original_grid[y][x] == ' ' ]

        for y in range(self.height):
            for x in range(self.width):
                cell = original_grid[y][x]
                pos = (x, y)

                is_horizontally_stuck = (x > 0 and x < self.width - 1 and original_grid[y][x-1] == '%' and original_grid[y][x+1] == '%')
                is_vertically_stuck = (y > 0 and y < self.height - 1 and original_grid[y-1][x] == '%' and original_grid[y+1][x] == '%')
                is_stuck = is_horizontally_stuck and is_vertically_stuck

                if cell == 'P':
                    self.initial_position = pos
                    self.grid[y][x] = ' '
                elif cell == 'G':
                    self.ghost_positions.append(pos)
                    self.grid[y][x] = ' '
                elif cell == 'E':
                    self.exit_position = pos
                elif cell in ['.', 'O']:
                    target_set = self.dots if cell == '.' else self.pies
                    if is_stuck:
                        if not empty_spaces:
                            raise ValueError(f"Cannot relocate stuck item '{cell}' at {pos}, no empty space left!")
                        new_pos = empty_spaces.pop(0)
                        target_set.add(new_pos)
                        # print(f"Relocated stuck item '{cell}' from {pos} to {new_pos}") # Removed
                    else:
                        target_set.add(pos)
                    self.grid[y][x] = ' '

        if self.initial_position == (-1, -1): raise ValueError("No initial position (P) found")
        if self.exit_position == (-1, -1): raise ValueError("No exit position (E) found")

        # Initialize an empty dictionary for lazy distance calculation
        self.distances = {}
        # print("Maze initialized. Distances will be calculated on-demand.") # Removed

    def _calculate_distances_from(self, start_node):
        """
        Đây là kỹ thuật "tính toán lười" (lazy calculation).
        Thay vì tính trước tất cả khoảng cách, hàm này chỉ được gọi khi cần.
        Nó sẽ chạy BFS từ 'start_node' đến tất cả các điểm khác và lưu kết quả
        vào self.distances để những lần sau chỉ cần tra cứu lại cho nhanh.
        """
        if start_node in self.distances:
            return

        self.distances[start_node] = {}
        q = deque([(start_node, 0)])
        visited = {start_node}
        self.distances[start_node][start_node] = 0

        while q:
            (x, y), dist = q.popleft()
            for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                neighbor = (x + dx, y + dy)
                if (0 <= neighbor[0] < self.width and 0 <= neighbor[1] < self.height and
                        not self.isWall(neighbor) and neighbor not in visited):
                    visited.add(neighbor)
                    self.distances[start_node][neighbor] = dist + 1
                    q.append((neighbor, dist + 1))

    def get_distance(self, pos1, pos2):
        """
        Hàm này trả về khoảng cách đường đi ngắn nhất giữa hai điểm.
        Nó sẽ kiểm tra xem khoảng cách đã được tính và lưu trữ chưa.
        Nếu chưa, nó sẽ gọi _calculate_distances_from để tính.
        """
        pos1, pos2 = (int(pos1[0]), int(pos1[1])), (int(pos2[0]), int(pos2[1]))

        # Thực hiện tính khoảng cách từ pos1 nếu chưa tính
        if pos1 not in self.distances:
            self._calculate_distances_from(pos1)

        # Thực hiện tính khoảng cách từ pos1 nếu chưa tính
        if pos2 not in self.distances:
            self._calculate_distances_from(pos2)

        # Kiểm tra khoảng cách từ pos1 đến pos2
        dist1 = self.distances.get(pos1, {}).get(pos2, float('inf'))

        # Kiểm tra khoảng cách từ pos2 đến pos1 (sử dụng tính chất đối xứng)
        dist2 = self.distances.get(pos2, {}).get(pos1, float('inf'))

        return min(dist1, dist2)

    def getGrid(self): return self.grid
    def getSize(self): return (self.width, self.height)
    def getInitPos(self): return self.initial_position
    def getDots(self): return self.dots.copy()
    def getPies(self): return self.pies.copy()
    def getGhosts(self): return self.ghost_positions.copy()
    def getExit(self): return self.exit_position

    def isWall(self, x, y=None):
        if y is None: x, y = x
        x, y = int(x), int(y)
        if not (0 <= y < self.height and 0 <= x < self.width): return True
        return self.grid[y][x] == "%"

#Lớp Problem

Lớp logic chính của trò chơi, có chức năng:

- **Quản lý trạng thái trò chơi** và **các bước chuyển trạng thái**  
- **Cài đặt thuật toán tìm kiếm A\*** và các thành phần liên quan  
- **Xây dựng các hàm heuristic (ước lượng)** để dẫn hướng cho quá trình tìm kiếm  
- **Xử lý các cơ chế đặc biệt của trò chơi**, bao gồm: *teleportation, pie effects, ghost movement, maze rotation*.


In [30]:
DIRECTIONS = {'North':(0,-1), 'South':(0,1), 'East':(1,0), 'West':(-1,0), 'Wait':(0,0)}
"""
    Đây là lớp định nghĩa bài toán tìm kiếm cho A*.
    Nó biết trạng thái bắt đầu, cách tạo ra các trạng thái kế tiếp từ một hành động,
    và làm thế nào để kiểm tra xem đã đến đích hay chưa.
    Nó cũng quản lý toàn bộ logic phức tạp về việc xoay mê cung.
"""

class Problem:
    def __init__(self, file_path):
        self.maze = Maze(file_path)
        initial_ghost_dirs = {i: 'East' for i in range(len(self.maze.getGhosts()))}
        self.initial_state = State(
            pos=self.maze.getInitPos(),
            dots=self.maze.getDots(),
            pies=self.maze.getPies(),
            ghost_positions=self.maze.getGhosts(),
            ghost_directions=initial_ghost_dirs,
            pie_steps=0,
            can_tele=True
        )
        self.heuristic_func = self.heuristic_custom
        self.rotated_grids = {0: self.maze.grid}
        self.rotated_dims = {0: (self.maze.height, self.maze.width)}

        # Pre-calculate rotated grids for first 4 rotations
        for i in range(1, 5):
            self.get_rotated_grid_and_dims(i)

    def get_rotated_grid_and_dims(self, num_rotations):
        """
        Trả về ma trận xoay và kích thước của nó.
        """
        num_rotations = num_rotations % 4

        if num_rotations not in self.rotated_grids:
            grid = self.maze.grid
            for _ in range(num_rotations):
                grid = np.rot90(grid, k=-1)  # Rotate 90° counterclockwise
            self.rotated_grids[num_rotations] = grid

        # Get actual dimensions after rotation
        h, w = self.rotated_grids[num_rotations].shape
        self.rotated_dims[num_rotations] = (h, w)

        return self.rotated_grids[num_rotations], (h, w)

    def get_initial_state(self): return self.initial_state

    def get_actions(self, state, current_rotation_level):
        """
        Lấy các hành động hợp lệ. 
        Nếu ở vị trí góc CÓ THỂ ĐI BỘ và có thể dịch chuyển tức thời, CHỈ cho phép các hành động dịch chuyển tức thời.
        """
        # 1. Xác định tọa độ các ô teleport WALKABLE dựa trên kích thước GỐC (w0, h0)
        h0, w0 = self.rotated_dims[0] # Original height and width
        if w0 < 2 or h0 < 2:
            return list(DIRECTIONS.keys()) # Maze quá nhỏ, không có ô teleport hợp lệ

        teleport_spots_orig = [
            (1, 1),                 # Có thể đi bộ ở góc trên bên trái
            (w0 - 2, 1),            # Có thể đi bộ ở góc trên bên phải
            (1, h0 - 2),            # Có thể đi bộ ở góc dưới bên trái
            (w0 - 2, h0 - 2)        # Có thể đi bộ ở góc dưới bên phải
        ]

        # 2. KIỂM TRA ĐIỀU KIỆN TELEPORT
        pos_rotated = state.get_pos()
        pos_original = self.unapply_rotations(pos_rotated, current_rotation_level)
        is_at_teleport_spot = pos_original in teleport_spots_orig

        # 3. Trả về hành động tương ứng
        if state.get_canTele() and is_at_teleport_spot:
            actions = [f'Teleport_{c[0]}_{c[1]}' for c in teleport_spots_orig if pos_original != c]
            return actions if actions else ['Wait']
        else:
            normal_actions = list(DIRECTIONS.keys())
            if 'Wait' in normal_actions:
                normal_actions.remove('Wait')
            return normal_actions

    def _get_next_ghost_state(self, ghost_positions_tuple, ghost_directions_dict, grid, grid_dims):
        """
            Hàm này tính toán nước đi tiếp theo cho tất cả các con ma.
            Quy tắc di chuyển của ma rất đơn giản:
            - Chúng chỉ di chuyển theo chiều ngang (Đông hoặc Tây).
            - Nếu đi tiếp được theo hướng hiện tại, chúng sẽ đi.
            - Nếu đụng tường, chúng sẽ quay đầu lại.
            - Nếu cả hai hướng đều bị chặn, chúng đứng yên.
        """
        new_directions = ghost_directions_dict.copy()
        h, w = grid_dims
        next_pos_list = [None] * len(ghost_positions_tuple)
        horizontal_directions = ['East', 'West']

        def is_wall_local(px, py):
             if not (0 <= py < h and 0 <= px < w): return True
             try: return grid[int(py)][int(px)] == '%'
             except IndexError: return True

        for i, pos in enumerate(ghost_positions_tuple):
            if i not in new_directions:
                next_pos_list[i] = pos # Ma đứng yên nếu không có hướng
                print(f"Warning: No direction found for ghost index {i}.")
                continue

            current_direction_name = new_directions[i]

            valid_horizontal_moves = {}
            for direction_name in horizontal_directions:
                dx, dy = DIRECTIONS[direction_name]
                neighbor_pos = (pos[0] + dx, pos[1] + dy)
                if not is_wall_local(neighbor_pos[0], neighbor_pos[1]):
                    valid_horizontal_moves[direction_name] = neighbor_pos

            next_direction_name = current_direction_name
            can_move_current = current_direction_name in valid_horizontal_moves

            if not can_move_current:
                opposite_horizontal = 'West' if current_direction_name == 'East' else 'East'
                if opposite_horizontal in valid_horizontal_moves:
                    next_direction_name = opposite_horizontal
                else:
                    next_direction_name = 'Wait'

            new_directions[i] = next_direction_name

            if next_direction_name == 'Wait':
                next_pos_list[i] = pos
            else:
                next_pos_list[i] = valid_horizontal_moves[next_direction_name]

        return tuple(next_pos_list), new_directions

    def rotate_point(self, pos, current_rotation_level):
        """Hàm tìm tọa độ mới của một điểm sau khi xoay 90 độ."""
        x, y = pos
        next_rotation = (current_rotation_level + 1) % 4

        _, (h_new, w_new) = self.get_rotated_grid_and_dims(next_rotation)

        x_new = w_new - 1 - y
        y_new = x

        return (int(x_new), int(y_new))

    def unrotate_point(self, pos, rotation_level):
        """Hàm tìm tọa độ gốc của một điểm đã bị xoay."""
        x_prime, y_prime = pos
        rotation = rotation_level % 4

        _, (h_from, w_from) = self.get_rotated_grid_and_dims(rotation_level)

        x_original = y_prime
        y_original = w_from - 1 - x_prime

        return (int(x_original), int(y_original))

    def apply_rotations(self, pos, num_rotations):
        """Áp dụng nhiều phép quay cho một điểm"""
        for i in range(num_rotations):
            pos = self.rotate_point(pos, i)
        return pos

    def apply_rotations_to_set(self, positions, num_rotations):
        """Áp dụng phép quay cho một tập hợp các vị trí"""
        return {self.apply_rotations(pos, num_rotations) for pos in positions}

    def unapply_rotations(self, pos, num_rotations):
        """Đảo ngược nhiều vòng quay"""
        for i in range(num_rotations, 0, -1):
            pos = self.unrotate_point(pos, i)
        return pos

    def unapply_rotations_to_set(self, positions, num_rotations):
        """Đảo ngược vòng quay cho một tập hợp các vị trí"""
        return {self.unapply_rotations(pos, num_rotations) for pos in positions}

    def get_neighbor_state(self, state, action, for_search_steps, check_ghost_collisions=True):
        """
            Hàm quan trọng nhất, mô phỏng một bước đi trong game.
            Nó nhận trạng thái hiện tại và một hành động (vd: 'North'), sau đó tính toán
            và trả về trạng thái mới của game sau khi hành động đó được thực hiện.
            Hàm này xử lý tất cả mọi thứ:
            1. Vị trí mới của Pacman.
            2. Vị trí mới của ma.
            3. Kiểm tra va chạm giữa Pacman và ma.
            4. Cập nhật lại danh sách chấm và bánh (nếu Pacman ăn chúng).
            5. Giảm bộ đếm năng lượng (pie_steps).
        """
        current_step_number = for_search_steps - 1
        current_rotation_level = current_step_number // 30
        rotation_key = current_rotation_level % 4
        grid, (h, w) = self.get_rotated_grid_and_dims(rotation_key)

        def is_wall_local(px, py):
            if not (0 <= py < h and 0 <= px < w): return True
            try:
                return grid[int(py)][int(px)] == '%'
            except IndexError:
                print(f"Warning: IndexError accessing grid at ({px}, {py}) with dims ({h}x{w}) at step {for_search_steps}")
                return True

        # --- 1. Xác định vị trí tiếp theo theo dự kiến của Pacman ---
        current_pacman_pos = state.get_pos()
        if action.startswith('Teleport'):
            tx_orig, ty_orig = map(int, action.split('_')[1:])
            target_spot_orig = (tx_orig, ty_orig)
            next_pacman_pos = self.apply_rotations(target_spot_orig, current_rotation_level)
            next_can_tele = False
        else:
            dx, dy = DIRECTIONS[action]
            next_pacman_pos = (current_pacman_pos[0] + dx, current_pacman_pos[1] + dy)
            next_can_tele = True
            if is_wall_local(next_pacman_pos[0], next_pacman_pos[1]):
                if state.get_pieSteps() == 0:
                    return None
                pass

        # --- 2. Xác định trạng thái của Ma cho bước kế tiếp ---
        next_step_number = for_search_steps
        current_ghost_positions_tuple = state.ghost_positions
        current_ghost_directions = state.get_ghost_directions()

        if next_step_number % 30 == 0:
            next_ghost_positions = current_ghost_positions_tuple
            next_ghost_directions = current_ghost_directions
        else:
            next_ghost_positions_tuple, next_ghost_directions_dict = self._get_next_ghost_state(
                current_ghost_positions_tuple, current_ghost_directions, grid, (h, w))
            next_ghost_positions = next_ghost_positions_tuple
            next_ghost_directions = next_ghost_directions_dict

        # --- 3. Kiểm tra va chạm ---
        if check_ghost_collisions:
            if state.pie_steps == 0:
                if next_pacman_pos in next_ghost_positions:
                    return None
                pacman_moves_onto_ghost_current = next_pacman_pos in current_ghost_positions_tuple
                ghost_moves_onto_pacman_current = current_pacman_pos in next_ghost_positions
                if pacman_moves_onto_ghost_current and ghost_moves_onto_pacman_current:
                    return None

        # --- 4. Cập nhật Dots và Pies ---
        current_dots = state.get_dots()
        current_pies = state.get_pies()
        next_dots = set(current_dots)
        next_pies = set(current_pies)
        next_pie_steps = max(0, state.get_pieSteps() - 1)

        if next_pacman_pos in next_dots:
            next_dots.remove(next_pacman_pos)
        elif next_pacman_pos in next_pies:
            next_pies.remove(next_pacman_pos)
            next_pie_steps = 5

        # --- 5. Tạo đối tượng trạng thái tiếp theo cuối cùng ---
        return State(
            pos=next_pacman_pos,
            dots=frozenset(next_dots),
            pies=frozenset(next_pies),
            ghost_positions=next_ghost_positions,
            ghost_directions=next_ghost_directions,
            pie_steps=next_pie_steps,
            can_tele=next_can_tele
        )

    def rotate_exit_point(self, num_rotations):
        """Tính toán vị trí cổng thoát sau khi quay"""
        exit_pos = self.maze.getExit()
        return self.apply_rotations(exit_pos, num_rotations)

    def is_goal_state(self, state, current_step_number):
        """
            Kiểm tra xem đã hoàn thành game chưa.
            Điều kiện thắng:
            1. Đã ăn hết tất cả các chấm (dots).
            2. Pacman đang đứng ở vị trí cổng thoát (Exit).
            Lưu ý: Vị trí Exit cũng bị xoay theo mê cung, nên phải so sánh với
            vị trí Exit đã được xoay tương ứng.
        """
        if state is None: return False
        if not state.get_dots():
            current_rotation_level = current_step_number // 30
            expected_exit_pos = self.rotate_exit_point(current_rotation_level)
            return state.get_pos() == expected_exit_pos
        return False

    def rotate_state(self, state, current_rotations):
        """
        Khi đến bước thứ 30 (hoặc 60, 90,...), hàm này được gọi.
        Nó sẽ xoay MỌI THỨ trong game: vị trí Pacman, vị trí ma, vị trí các chấm, bánh.
        Nó tạo ra một State hoàn toàn mới tương ứng với mê cung đã bị xoay.
        """
        if state is None:
            return None

        new_pos = self.rotate_point(state.pos, current_rotations)

        new_ghost_positions_list = [self.rotate_point(gp, current_rotations)
                                      for gp in state.ghost_positions]
        new_ghost_positions_tuple = tuple(new_ghost_positions_list)

        new_dots = {self.rotate_point(dp, current_rotations)
                      for dp in state.dots}
        new_pies = {self.rotate_point(pp, current_rotations)
                      for pp in state.pies}

        return State(new_pos, new_dots, new_pies,
                     new_ghost_positions_tuple,
                     state.get_ghost_directions(),
                     state.pie_steps,
                     state.can_tele)

    def get_unrotated_for_heuristic(self, state, current_step_number):
        """Quay lại vị trí ban đầu (Xoay 0)"""
        current_rotation_level = current_step_number // 30
        pos_orig = self.unapply_rotations(state.pos, current_rotation_level)
        dots_orig = self.unapply_rotations_to_set(state.dots, current_rotation_level)
        ghosts_orig = self.unapply_rotations_to_set(state.ghost_positions, current_rotation_level)
        return pos_orig, dots_orig, ghosts_orig

    def heuristic_custom(self, state, current_step_number):
        """
        Hàm heuristic cho thuật toán A*. Ước tính chi phí từ trạng thái hiện tại đến đích.

        --------------------------------------------------------------------------------
        **THẢO LUẬN VỀ HEURISTIC**

        Hàm heuristic này được đề xuất để đáp ứng yêu cầu không sử dụng khoảng cách
        Euclid hay Manhattan. Thay vào đó, nó dựa trên khoảng cách di chuyển thực tế
        trong mê cung (được tính bằng thuật toán BFS và lưu lại để tối ưu tốc độ).

        Ý tưởng chính là ước tính chi phí còn lại bằng cách mô phỏng một chiến lược
        tham lam (greedy):
        1. Từ vị trí hiện tại của Pacman, tìm điểm thức ăn (dot) gần nhất.
        2. Cộng dồn khoảng cách di chuyển tới điểm đó.
        3. Cập nhật vị trí hiện tại là điểm vừa ăn, sau đó lặp lại cho đến khi hết chấm.
        4. Cuối cùng, cộng thêm khoảng cách từ điểm cuối cùng đến cổng thoát (Exit).

        --------------------------------------------------------------------------------
        **1. TÍNH CHẤP NHẬN ĐƯỢC (Admissibility)**

        - **Kết luận:** Hàm heuristic này **LÀ CHẤP NHẬN ĐƯỢC (admissible)**.
        - **Giải thích:** Một heuristic được gọi là chấp nhận được nếu nó không bao giờ
          đánh giá chi phí cao hơn chi phí thực tế để đến đích (h(n) <= h*(n)).
          Hàm này thỏa mãn điều kiện vì:
            a. Nó sử dụng khoảng cách BFS, là đường đi ngắn nhất thực tế trong một
               mê cung tĩnh (không có ma, không xoay).
            b. Chiến lược "ăn chấm gần nhất" chỉ là một trong nhiều cách để ăn hết chấm,
               và không phải lúc nào cũng là cách tối ưu nhất. Do đó, chi phí mà nó
               ước tính sẽ luôn nhỏ hơn hoặc bằng chi phí của lộ trình tối ưu thực sự.
            c. Heuristic này bỏ qua các yếu tố làm tăng chi phí trong thực tế như
               việc phải đi đường vòng để né ma hoặc sự thay đổi của mê cung khi xoay.
          Vì những lý do trên, nó không bao giờ "bi quan" hơn thực tế.

        --------------------------------------------------------------------------------
        **2. TÍNH NHẤT QUÁN (Consistency / Monotonicity)**

        - **Kết luận:** Hàm heuristic này **KHÔNG NHẤT QUÁN (not consistent)**.
        - **Giải thích:** Một heuristic được gọi là nhất quán nếu với mọi node n và
          node kề sau n' của nó, ta có h(n) <= c(n, n') + h(n'), với c(n, n') là chi
          phí thực tế để đi từ n đến n'.
          Hàm này có thể vi phạm điều kiện trên vì chiến lược "ăn chấm gần nhất":
            - Ví dụ: Tại trạng thái n, chấm gần nhất là A. Heuristic tính toán dựa trên
              việc đi đến A.
            - Sau khi Pacman di chuyển một bước đến trạng thái n' (với chi phí c(n,n')=1),
              chấm gần nhất có thể đột ngột thay đổi thành một chấm B ở rất xa.
            - Sự thay đổi đột ngột này có thể khiến giá trị heuristic giảm đi một lượng
              lớn, lớn hơn chi phí 1 bước đi, dẫn đến h(n) > 1 + h(n').
          Tuy nhiên, việc không nhất quán không phải là vấn đề lớn. Miễn là heuristic
          chấp nhận được, thuật toán A* vẫn đảm bảo tìm ra đường đi tối ưu, dù có thể
          phải duyệt lại một số node đã từng duyệt.
        --------------------------------------------------------------------------------
        """
        if state is None: return float('inf')

        pos_orig, dots_orig, ghosts_orig = self.get_unrotated_for_heuristic(state, current_step_number)

        if not dots_orig:
            return self.maze.get_distance(pos_orig, self.maze.getExit())

        greedy_cost = self._calculate_greedy_path_cost(pos_orig, dots_orig, self.maze.getExit())

        teleport_cost = float('inf')
        h0, w0 = self.rotated_dims[0]
        if w0 >= 4 and h0 >= 4:
            teleport_spots_orig = [
                (1, 1), (w0 - 2, 1),
                (1, h0 - 2), (w0 - 2, h0 - 2)
            ]

            cost_to_tele_entry = float('inf')
            nearest_tele_entry = None
            if pos_orig in teleport_spots_orig:
                cost_to_tele_entry = 0
                nearest_tele_entry = pos_orig
            else:
                try:
                    nearest_tele_entry = min(teleport_spots_orig, key=lambda t: self.maze.get_distance(pos_orig, t))
                    cost_to_tele_entry = self.maze.get_distance(pos_orig, nearest_tele_entry)
                except (ValueError, TypeError):
                    cost_to_tele_entry = float('inf')

            if cost_to_tele_entry != float('inf'):
                best_heuristic_after_teleport = float('inf')

                for tele_exit in teleport_spots_orig:
                    if tele_exit == nearest_tele_entry: continue

                    h_from_exit = self._calculate_greedy_path_cost(tele_exit, dots_orig, self.maze.getExit())
                    best_heuristic_after_teleport = min(best_heuristic_after_teleport, h_from_exit)

                if best_heuristic_after_teleport != float('inf'):
                    teleport_cost = cost_to_tele_entry + 1 + best_heuristic_after_teleport

        total_distance = min(greedy_cost, teleport_cost)

        ghost_penalty = 0
        if state.get_pieSteps() == 0 and ghosts_orig:
            min_dist_to_ghost = min((self.maze.get_distance(pos_orig, gp) for gp in ghosts_orig), default=float('inf'))

            if min_dist_to_ghost == 0: ghost_penalty = 5000
            elif min_dist_to_ghost < 2: ghost_penalty = 1000 / (min_dist_to_ghost + 0.1)
            elif min_dist_to_ghost < 4: ghost_penalty = 100 / (min_dist_to_ghost + 0.1)

        return total_distance + ghost_penalty

    def _calculate_greedy_path_cost(self, start_pos, dots, exit_pos):
        """Hàm trợ giúp tính toán chi phí đường đi tham lam (ăn điểm gần nhất)."""
        total_distance = 0
        remaining_dots = list(dots)
        current_pos = start_pos

        while remaining_dots:
            nearest_dot = min(remaining_dots, key=lambda d: self.maze.get_distance(current_pos, d))
            dist_to_dot = self.maze.get_distance(current_pos, nearest_dot)
            if dist_to_dot == float('inf'): return float('inf')

            total_distance += dist_to_dot
            current_pos = nearest_dot
            remaining_dots.remove(nearest_dot)

        dist_to_exit = self.maze.get_distance(current_pos, exit_pos)
        if dist_to_exit == float('inf'): return float('inf')

        total_distance += dist_to_exit
        return total_distance

#**Triển khai thuật toán A\* search**

In [31]:
class AStarSolver:
    def __init__(self, problem, heuristic):
        # problem: đối tượng chứa định nghĩa 8-puzzle (initial state, goal, successor,...)
        # heuristic: hàm ước lượng (misplaced tiles, manhattan, ...)
        self.problem = problem
        self.heuristic = heuristic

    def search(self):
        """Thuật toán A* tìm đường đi ngắn nhất từ trạng thái đầu đến trạng thái đích."""

        # --- Khởi tạo ---
        start = self.problem.initial_state                     # Lấy trạng thái bắt đầu
        open_list = [Node(start, g=0, h=self.heuristic(start))] # Hàng đợi ưu tiên chứa node ban đầu
        heapq.heapify(open_list)                               # Dùng heap để tự động sắp xếp theo f = g + h
        closed = {}                                            # Lưu trạng thái đã duyệt (state -> chi phí tốt nhất)
        expanded = 0                                           # Đếm số node đã mở rộng (phục vụ thống kê)

        # --- Vòng lặp tìm kiếm chính ---
        while open_list:
            # Lấy node có giá trị f nhỏ nhất trong open_list
            node = heapq.heappop(open_list)

            # Nếu đã có trong closed với chi phí tốt hơn, bỏ qua
            if node.state in closed and closed[node.state] <= node.g:
                continue

            # Cập nhật node hiện tại vào closed (đã thăm)
            closed[node.state] = node.g
            expanded += 1  # Tăng bộ đếm node đã mở rộng

            # --- Kiểm tra điều kiện dừng ---
            if self.problem.is_goal(node.state):
                # Nếu đạt trạng thái mục tiêu, trả về đường đi, chi phí, và số node đã mở rộng
                return node.reconstruct_path(), node.g, expanded

            # --- Sinh các trạng thái kế tiếp ---
            for action, succ in self.problem.get_successors(node.state):
                # g_new = chi phí đi đến trạng thái kế tiếp (ở đây = g + 1)
                g_new = node.g + self.problem.cost(node.state, succ)

                # Nếu trạng thái này đã được duyệt với chi phí tốt hơn thì bỏ qua
                if succ in closed and closed[succ] <= g_new:
                    continue

                # Thêm node mới vào open_list (hàng đợi ưu tiên)
                heapq.heappush(open_list, Node(
                    succ,          # trạng thái kế tiếp
                    node,          # node cha
                    action,        # hành động đã thực hiện (U/D/L/R)
                    g_new,         # chi phí thực tế đến đây
                    self.heuristic(succ)  # chi phí ước lượng còn lại
                ))

        # Nếu không tìm thấy đường đi (trường hợp vô nghiệm)
        return None, float('inf'), expanded

#Lớp GameVisualizer
-  **__init__(self, problem, solution=None):** Thiết lập Pygame, lưu trữ bài toán và giải pháp, định nghĩa màu sắc, tính toán kích thước ô lưới ban đầu và tạo cửa sổ trò chơi.



- **_calculate_initial_cell_size(self):** Tính toán kích thước pixel lớn nhất có thể cho mỗi ô lưới sao cho mê cung (dựa trên kích thước xoay lớn nhất) vừa vặn trên màn hình.



- **resize_window(self):** Cập nhật kích thước cửa sổ Pygame (Width x Height) để khớp với kích thước của mê cung ở cấp độ xoay hiện tại (xảy ra sau mỗi 30 bước di chuyển).


- **show_integrated_menu(self):** Vẽ trạng thái ban đầu của mê cung, sau đó phủ menu lên trên để người dùng chọn giữa **Chế độ Tự động (A)** hoặc **Chế độ Thủ công (M)**.



- **draw_maze(self, current_state):** Xóa màn hình và vẽ lại toàn bộ mê cung, bao gồm: **tường**, **điểm thoát**, **dot**, **power pie**, **Pacman** và **Ghost** (với màu sắc tương ứng trạng thái sợ hãi). Đồng thời vẽ thông tin chỉ số (step, dot, power, rotation).



- **run_auto_game(self):** Mô phỏng từng bước theo chuỗi hành động đã giải được (`solution_path`). Nó tính toán và lưu trữ các trạng thái, bao gồm cả hiệu ứng xoay, và hiển thị chúng theo tốc độ `AUTO_MODE_FPS`.



- **run_manual_game(self):** Chờ đợi input từ người dùng (phím mũi tên) để di chuyển Pacman. Nó quản lý quá trình dịch chuyển tức thời (teleport), hiệu ứng xoay, và kiểm tra điều kiện thắng/thua (va chạm Ghost hoặc đạt mục tiêu).

In [32]:
class GameVisualizer:
    """
        Lớp này chịu trách nhiệm cho mọi thứ liên quan đến đồ họa.
        Nó lấy logic game từ lớp Problem và "vẽ" nó ra màn hình bằng Pygame.
        Việc tách riêng logic và đồ họa giúp code rất sạch sẽ.
    """
    def __init__(self, problem, solution=None):
        pygame.init()
        self.problem = problem
        self.solution_path = solution if solution else []
        self.total_steps = len(self.solution_path) if solution else "N/A"

        self.BLACK = (0, 0, 0)
        self.WHITE = (255, 255, 255)
        self.YELLOW = (255, 255, 0)
        self.BLUE = (0, 0, 255)
        self.RED = (255, 0, 0)
        self.GREEN = (0, 255, 0)
        self.PURPLE = (128, 0, 128)
        self.SCARED_GHOST_BLUE = (60, 60, 255)

        self.step_count = 0
        self.AUTO_MODE_FPS = 4
        self.MANUAL_MODE_FPS = 10

        self.cell_size = self._calculate_initial_cell_size()

        initial_h, initial_w = self.problem.rotated_dims[0]
        self.width = initial_w * self.cell_size
        self.height = initial_h * self.cell_size

        self.screen = None
        try:
            self.screen = pygame.display.set_mode((self.width, self.height))
            pygame.display.set_caption(f"Pacman - Initializing")
        except pygame.error as e:
            print(f"Fatal Error: Could not create Pygame window ({self.width}x{self.height}): {e}")
            raise SystemExit(f"Pygame window creation failed: {e}")

    def _calculate_initial_cell_size(self):
        """Trợ giúp tính toán kích thước ô dựa trên màn hình và kích thước mê cung tối đa có thể."""
        try:
            screen_info = pygame.display.Info()
            screen_w, screen_h = screen_info.current_w, screen_info.current_h
            if screen_w <= 0 or screen_h <= 0:
                raise pygame.error("Invalid screen dimensions reported.")
        except pygame.error as e:
             print(f"Warning: Could not get screen info ({e}). Using default screen size (1200x800).")
             screen_w, screen_h = 1200, 800

        if 0 not in self.problem.rotated_dims or 1 not in self.problem.rotated_dims:
              print("Error: Initial or rotated maze dimensions not pre-calculated.")
              return 15

        h0, w0 = self.problem.rotated_dims[0]
        h1, w1 = self.problem.rotated_dims[1]

        max_maze_w = max(w0, w1)
        max_maze_h = max(h0, h1)

        if max_maze_w <= 0 or max_maze_h <= 0:
              print(f"Error: Invalid max maze dimensions ({max_maze_h}x{max_maze_w}).")
              return 15

        screen_fraction = 0.90
        cell_w_calc = (screen_w * screen_fraction) // max_maze_w
        cell_h_calc = (screen_h * screen_fraction) // max_maze_h

        cell_size = max(15, int(min(cell_w_calc, cell_h_calc)))
        return cell_size

    def resize_window(self):
        """Thay đổi kích thước cửa sổ dựa trên kích thước mê cung hiện tại sau khi xoay."""
        current_step = self.step_count
        current_rotation_level = current_step // 30
        rotation_key = current_rotation_level % 4

        if rotation_key not in self.problem.rotated_dims:
              print(f"Error: Rotated dimensions for key {rotation_key} not found.")
              return
        h, w = self.problem.rotated_dims[rotation_key]
        if w <= 0 or h <= 0:
              print(f"Error: Invalid dimensions ({h}x{w}) for rotation {rotation_key}.")
              return

        new_width = w * self.cell_size
        new_height = h * self.cell_size

        if new_width != self.width or new_height != self.height:
            self.width = new_width
            self.height = new_height
            try:
                self.screen = pygame.display.set_mode((self.width, self.height))
            except pygame.error as e:
                print(f"Error resizing window: {e}")
                self.screen = None
                raise SystemExit(f"Pygame window resize failed: {e}")

        if self.screen:
             pygame.display.set_caption(f"Pacman - Step {self.step_count} - {w}x{h}")

    def show_integrated_menu(self):
        """Vẽ trạng thái mê cung ban đầu và chồng lên menu lựa chọn chế độ."""
        if not self.screen:
            print("Cannot show menu - screen not initialized.")
            return None

        initial_state = self.problem.get_initial_state()
        self.step_count = 0
        self.draw_maze(initial_state)

        try:
            if not pygame.font.get_init(): pygame.font.init()
            font_large = pygame.font.Font(None, min(70, self.height // 8))
            font_medium = pygame.font.Font(None, min(50, self.height // 12))
            font_small = pygame.font.Font(None, min(30, self.height // 20))
        except Exception as e:
            print(f"Warning: Font loading error ({e}). Using default system font.")
            font_large = pygame.font.SysFont(None, 70)
            font_medium = pygame.font.SysFont(None, 50)
            font_small = pygame.font.SysFont(None, 30)

        title_text = font_large.render("Pacman AI", True, self.YELLOW)
        auto_text = font_medium.render("Press 'A' for Auto Mode (A*)", True, self.WHITE)
        manual_text = font_medium.render("Press 'M' for Manual Mode", True, self.WHITE)
        quit_text = font_small.render("Press ESC to Quit", True, self.WHITE)

        overlay = pygame.Surface((self.width, self.height), pygame.SRCALPHA)
        overlay.fill((0, 0, 0, 192))

        title_rect = title_text.get_rect(center=(self.width / 2, self.height * 0.3))
        auto_rect = auto_text.get_rect(center=(self.width / 2, self.height * 0.5))
        manual_rect = manual_text.get_rect(center=(self.width / 2, self.height * 0.65))
        quit_rect = quit_text.get_rect(center=(self.width / 2, self.height * 0.85))

        overlay.blit(title_text, title_rect)
        overlay.blit(auto_text, auto_rect)
        overlay.blit(manual_text, manual_rect)
        overlay.blit(quit_text, quit_rect)
        self.screen.blit(overlay, (0, 0))
        pygame.display.flip()

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return None
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_a:
                        return "AUTO"
                    if event.key == pygame.K_m:
                        return "MANUAL"
                    if event.key == pygame.K_ESCAPE:
                        return None
            pygame.time.Clock().tick(30)

    def draw_maze(self, current_state):
        """Vẽ trạng thái trò chơi hiện tại"""
        if not self.screen: return
        self.screen.fill(self.BLACK)

        current_rotations = self.step_count // 30
        rotation_key = current_rotations % 4
        grid, (h, w) = self.problem.get_rotated_grid_and_dims(rotation_key)

        exit_pos_rotated = self.problem.rotate_exit_point(current_rotations)

        cell_size = self.cell_size
        half_cell = cell_size / 2

        for y in range(h):
            for x in range(w):
                if 0 <= y < grid.shape[0] and 0 <= x < grid.shape[1] and grid[y][x] == '%':
                    pygame.draw.rect(self.screen, self.BLUE,
                                     (x * cell_size, y * cell_size, cell_size, cell_size), 2)
        if exit_pos_rotated and 0 <= exit_pos_rotated[0] < w and 0 <= exit_pos_rotated[1] < h:
            pygame.draw.rect(self.screen, self.PURPLE,
                             (exit_pos_rotated[0] * cell_size, exit_pos_rotated[1] * cell_size,
                              cell_size, cell_size))
        obj_radius = max(2, int(cell_size * 0.4))
        dot_radius = max(1, int(cell_size * 0.1))
        pie_radius = max(2, int(cell_size * 0.25))

        for dot_pos in current_state.dots:
            pygame.draw.circle(self.screen, self.WHITE, (int(dot_pos[0] * cell_size + half_cell), int(dot_pos[1] * cell_size + half_cell)), dot_radius)
        for pie_pos in current_state.pies:
            pygame.draw.circle(self.screen, self.GREEN, (int(pie_pos[0] * cell_size + half_cell), int(pie_pos[1] * cell_size + half_cell)), pie_radius)
        pacman_pos_rotated = current_state.pos
        pygame.draw.circle(self.screen, self.YELLOW, (int(pacman_pos_rotated[0] * cell_size + half_cell), int(pacman_pos_rotated[1] * cell_size + half_cell)), obj_radius)
        is_scared = current_state.pie_steps > 0
        ghost_color = self.SCARED_GHOST_BLUE if is_scared else self.RED
        for ghost_pos in current_state.ghost_positions:
            pygame.draw.circle(self.screen, ghost_color, (int(ghost_pos[0] * cell_size + half_cell), int(ghost_pos[1] * cell_size + half_cell)), obj_radius)

        try:
            font_ui = pygame.font.Font(None, max(20, int(cell_size * 0.8)))
        except:
            font_ui = pygame.font.SysFont(None, 24)

        # 1. Chuẩn bị chuỗi text cho Step tùy theo chế độ chơi
        if self.total_steps == "N/A":
            step_text = f"Step: {self.step_count}"
        else:
            step_text = f"Step: {self.step_count}/{self.total_steps}"
            
        # 2. Xác định các chuỗi thông tin theo thứ tự mong muốn
        info_texts = [
            step_text,
            f"Power: {current_state.pie_steps}",
            f"Rotation: {rotation_key}",
            f"Dots: {len(current_state.dots)}"
        ]
        
        num_items = len(info_texts)
        y_pos = 10  # Căn lề một chút so với cạnh trên của cửa sổ

        # 3. Lặp qua và đặt vị trí cho từng mục thông tin
        for i, text_str in enumerate(info_texts):
            text_surf = font_ui.render(text_str, True, self.WHITE)
            
            # Tính toán vị trí x để mỗi mục thông tin được căn giữa trong "ô" của nó
            x_pos = (i + 0.5) * (self.width / num_items)
            
            text_rect = text_surf.get_rect(center=(x_pos, y_pos))
            self.screen.blit(text_surf, text_rect)

        pygame.display.flip()

    def run_auto_game(self):
        """Chạy trò chơi ở chế độ tự động bằng cách sử dụng đường dẫn giải pháp được tính toán trước."""
        if not self.solution_path:
            print("Error: No solution path available for Auto Mode!")
            if self.screen:
                try:
                    font_error = pygame.font.Font(None, 40)
                    text_error = font_error.render("No solution found!", True, self.RED)
                    text_rect = text_error.get_rect(center=(self.width / 2, self.height / 2))
                    self.screen.fill(self.BLACK)
                    self.screen.blit(text_error, text_rect)
                    pygame.display.flip()
                    pygame.time.wait(3000)
                except Exception as e: print(f"Error displaying message: {e}")
            return

        state_history = [self.problem.get_initial_state()]
        current_vis_state = self.problem.get_initial_state()

        for i, action in enumerate(self.solution_path):
            step_num_for_next = i + 1
            next_vis_state_before_rot = self.problem.get_neighbor_state(current_vis_state, action, step_num_for_next)

            if next_vis_state_before_rot is None:
                print(f"Error: Solution path leads to invalid state at step {step_num_for_next} with action {action}.")
                print(f"Previous State: {current_vis_state}")
                break

            final_next_vis_state = next_vis_state_before_rot
            if step_num_for_next % 30 == 0:
                rotation_level_before = i // 30
                rotated_next_state = self.problem.rotate_state(final_next_vis_state, rotation_level_before)
                if rotated_next_state is None:
                    print(f"Error: State rotation failed at step {step_num_for_next}")
                    break
                final_next_vis_state = rotated_next_state

            state_history.append(final_next_vis_state)
            current_vis_state = final_next_vis_state

        running = True; paused = False
        clock = pygame.time.Clock()
        self.step_count = 0

        while running and self.step_count < len(state_history):
            if not self.screen:
                print("Error: Screen became unavailable.")
                running = False
                break

            for event in pygame.event.get():
                if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
                    running = False
                if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                    paused = not paused

            if not paused:
                current_playback_state = state_history[self.step_count]
                if self.step_count > 0 and self.step_count % 30 == 0:
                    self.resize_window()
                    if not self.screen: running = False; break
                self.draw_maze(current_playback_state)
                self.step_count += 1
            else:
                if self.step_count < len(state_history):
                    self.draw_maze(state_history[self.step_count])

            clock.tick(self.AUTO_MODE_FPS)

        if running and self.step_count >= len(state_history) and self.screen:
            try:
                font_end = pygame.font.Font(None, min(80, self.height // 8))
                text_end = font_end.render("You Win!", True, self.GREEN)
                text_rect = text_end.get_rect(center=(self.width / 2, self.height / 2))
                if state_history: 
                    self.step_count -= 1
                    self.draw_maze(state_history[-1])
                
                self.screen.blit(text_end, text_rect)
                pygame.display.flip()
                pygame.time.wait(3000)
            except Exception as e:
                print(f"Error displaying completion message: {e}")

    def run_manual_game(self):
        """
        Chạy trò chơi ở chế độ thủ công.
        """
        current_state = self.problem.get_initial_state()
        running = True
        clock = pygame.time.Clock()
        manual_step = 0

        h0, w0 = self.problem.rotated_dims[0]
        teleport_spots_orig = []
        if w0 >= 4 and h0 >= 4:
            teleport_spots_orig = [(1, 1), (w0 - 2, 1), (1, h0 - 2), (w0 - 2, h0 - 2)]

        while running:
            if not self.screen:
                print("Error: Screen unavailable.")
                break

            self.step_count = manual_step

            self.resize_window()
            if not self.screen: break
            self.draw_maze(current_state)

            action = None
            user_action_taken = False
            while not user_action_taken and running:
                pygame.event.pump()
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False; user_action_taken = True
                    if event.type == pygame.KEYDOWN:
                        user_action_taken = True
                        if event.key == pygame.K_ESCAPE: running = False
                        elif event.key == pygame.K_UP: action = 'North'
                        elif event.key == pygame.K_DOWN: action = 'South'
                        elif event.key == pygame.K_LEFT: action = 'West'
                        elif event.key == pygame.K_RIGHT: action = 'East'
                        elif event.key == pygame.K_SPACE: action = 'Wait'

                if not user_action_taken:
                    clock.tick(self.MANUAL_MODE_FPS)

            if not running or action is None or action == 'Wait':
                continue

            next_step_num = manual_step + 1

            next_state = self.problem.get_neighbor_state(
                current_state, action, for_search_steps=next_step_num, check_ghost_collisions=False)

            if next_state is None:
                continue

            current_state = next_state
            manual_step += 1

            pacman_pos = current_state.get_pos()
            ghost_positions = current_state.get_ghost_positions()
            if current_state.get_pieSteps() == 0 and pacman_pos in ghost_positions:
                print("\n=== GAME OVER! ===")
                self.draw_maze(current_state)
                try:
                    font_end = pygame.font.Font(None, min(80, self.height // 8))
                    text_end = font_end.render("Game Over!", True, self.RED)
                    text_rect = text_end.get_rect(center=(self.width / 2, self.height / 2))
                    self.screen.blit(text_end, text_rect)
                    pygame.display.flip()
                    pygame.time.wait(3000)
                except Exception as e:
                    print(f"Error displaying game over message: {e}")
                running = False
                continue

            pos_orig = self.problem.unapply_rotations(current_state.pos, manual_step // 30)
            is_at_teleport_spot = pos_orig in teleport_spots_orig

            if current_state.can_tele and is_at_teleport_spot:
                destinations = [spot for spot in teleport_spots_orig if spot != pos_orig]
                if destinations:
                    tele_target_orig = random.choice(destinations)
                    tele_action = f'Teleport_{tele_target_orig[0]}_{tele_target_orig[1]}'

                    tele_step_num = manual_step + 1
                    tele_state = self.problem.get_neighbor_state(
                        current_state, tele_action, tele_step_num, check_ghost_collisions=False)

                    if tele_state:
                        current_state = tele_state
                        manual_step += 1

            if manual_step % 30 == 0 and manual_step > 0:
                rotation_level_before = (manual_step - 1) // 30
                rotated_state = self.problem.rotate_state(current_state, rotation_level_before)
                if rotated_state:
                    current_state = rotated_state

            if self.problem.is_goal_state(current_state, manual_step):
                print("\n=== YOU WIN! ===")
                print(f"Completed in {manual_step} steps!")
                self.draw_maze(current_state)
                try:
                    font_end = pygame.font.Font(None, min(80, self.height // 8))
                    text_end = font_end.render("You Win!", True, self.GREEN)
                    text_rect = text_end.get_rect(center=(self.width / 2, self.height / 2))
                    self.screen.blit(text_end, text_rect)
                    pygame.display.flip()
                    pygame.time.wait(3000)
                except Exception as e:
                    print(f"Error displaying win message: {e}")
                running = False

            clock.tick(self.MANUAL_MODE_FPS)

### Hàm run_game(): Khởi chạy chính của trò chơi Pac-Man
- Chức năng:
    1. Khởi tạo Pygame.
    2. Tải mê cung từ file 'pacman.txt' và khởi tạo đối tượng Problem.
    3. Khởi tạo GameVisualizer và hiển thị menu chọn chế độ.
    4. **Chế độ AUTO:** Tạo adapter (PacmanProblemAdapter) và heuristic wrapper, khởi tạo và chạy thuật toán **AStarSolver** để tìm đường đi. Sau đó, hiển thị kết quả A* (chi phí, hành động) và chạy mô phỏng (run_auto_game).
    5. **Chế độ MANUAL:** Chạy chế độ điều khiển thủ công (run_manual_game).
    6. Bắt và xử lý các lỗi chính (File not found, Pygame error, SystemExit).
    7. Dọn dẹp môi trường bằng pygame.quit() ở cuối cùng.

 - if __name__ == "__main__":  Đảm bảo hàm run_game() chỉ được gọi khi file Python này được thực thi trực tiếp, không phải khi nó được import như một module.

In [33]:
def run_game():
    """
        Hàm chính, là điểm khởi đầu của toàn bộ chương trình.
        Nhiệm vụ của nó là:
        1. Khởi tạo Pygame.
        2. Đọc file mê cung và tạo ra đối tượng `Problem`.
        3. Tạo đối tượng `GameVisualizer` để xử lý đồ họa.
        4. Hiển thị menu và nhận lựa chọn của người chơi.
        5. Nếu chọn 'AUTO', nó sẽ gọi `a_star_search` để tìm lời giải.
        6. Cuối cùng, nó gọi hàm `run_auto_game` hoặc `run_manual_game` để bắt đầu game.
        7. Xử lý các lỗi có thể xảy ra và dọn dẹp Pygame khi kết thúc.
    """
    problem = None
    visualizer = None
    try:
        pygame.quit()
        pygame.init()

        maze_file = os.path.join(CURRENT_DIR, 'pacman.txt')
        if not os.path.exists(maze_file):
            raise FileNotFoundError(f"Error: Maze file not found at {maze_file}")

        problem = Problem(maze_file)
        visualizer = GameVisualizer(problem, None)
        mode = visualizer.show_integrated_menu()

        if mode is None:
            return

        if mode == "AUTO":
            # --- PHẦN SỬA ĐỔI ĐỂ TÁI SỬ DỤNG A* TỪ TASK 1 ---

            # 1. Tạo adapter cho bài toán Pac-Man để nó tương thích với AStarSolver
            pacman_adapter = PacmanProblemAdapter(problem)

            # 2. Tạo một hàm heuristic tương thích (chỉ nhận `state`)
            compatible_heuristic = heuristic_wrapper(problem)

            # 3. Khởi tạo và chạy AStarSolver từ Task 1
            # (Giả định class AStarSolver và Node của nó đã có trong file này)
            solver = AStarSolver(pacman_adapter, compatible_heuristic)
            solution, total_cost, _ = solver.search() # Gọi hàm search và nhận kết quả

            # --- KẾT THÚC PHẦN SỬA ĐỔI ---

            if not solution:
                print("\n❌ Failed to find solution within timeout.")
                if visualizer.screen:
                    try:
                        font_error = pygame.font.Font(None, 40)
                        text_error = font_error.render("A* could not find a solution!", True, visualizer.RED)
                        text_rect = text_error.get_rect(center=(visualizer.width / 2, visualizer.height / 2))
                        visualizer.screen.fill(visualizer.BLACK)
                        visualizer.screen.blit(text_error, text_rect)
                        pygame.display.flip()
                        pygame.time.wait(4000)
                    except Exception as e: print(f"Error displaying A* fail message: {e}")
                return

            # AStarSolver đã trả về total_cost, không cần tính lại bằng len(solution)
            output_actions = ['Stop' if action == 'Wait' else action for action in solution]

            print("\n" + "="*20)
            print("OUPUT")
            print("="*20)
            print(f"Total cost: {total_cost}")
            print(f"Action list: {output_actions}")
            print("="*20 + "\n")

            visualizer.solution_path = solution
            visualizer.total_steps = len(solution) # len(solution) vẫn đúng để hiển thị số bước
            visualizer.run_auto_game()

        elif mode == "MANUAL":
            visualizer.run_manual_game()

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except ValueError as e:
        print(f"Configuration Error: {e}")
    except pygame.error as e:
        print(f"Pygame Error: {e}")
        traceback.print_exc()
    except SystemExit as e:
        print(f"System exit requested: {e}")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred in run_game: {str(e)}")
        traceback.print_exc()
    finally:
        pygame.quit()
        
# Dòng này kiểm tra xem file có đang được chạy trực tiếp không.
# Nếu có, nó sẽ gọi hàm run_game() để bắt đầu.
if __name__ == "__main__":
    run_game()